# Tutorial for Cosmos DB and Python for the SQL API

As you probably know, [Cosmos DB](https://docs.microsoft.com/en-us/azure/cosmos-db/) is a globally distributed, multi-model database service in Azure that supports document, key-value, wide-column, and graph databases.

Cosmos DB includes support for multiple Software Development Kits for programming languages such as Python. Multiple resources already exist that help to learn to use the Python SDK: 

* [Cosmos DB reference guide for the Python SDK](https://docs.microsoft.com/en-us/python/api/azure-cosmos/?view=azure-python)
* [Cosmos DB samples with the Python SDK](https://github.com/Azure/azure-cosmos-python/blob/master/test/crud_tests.py)
* [EDX course for Cosmos DB](https://courses.edx.org/courses/course-v1:Microsoft+DAT237x+2T2017/course/): most of the examples in this book come from this EDX course, feel free to go to the course to deepen in the concepts presented in this notebook.


## Why a Jupyter Notebook?

As opposed to documentation or code samples, Jupyter Notebooks offer a very interesting combination of rich text for explanations and pieces of code that can be executed independently from each other, which makes it ideal to learn a new technology.

This notebook will go over the basic (and not so basic) concepts of using Cosmos DB with its Python SDK for the SQL API. You can run this repository from any system with Python 3.x, or using the free platform [Azure Notebooks](https://notebooks.azure.com/), where you have the option of importing Github repositories.

Feel free to try some different variations of the code in this notebook, to see different effects.



## Creating your Cosmos DB account

All Cosmos databases are contained in a Cosmos DB account. You can create a Cosmos DB account in Azure using any Azure mechanism. In this notebook we will assume that you have installed the [Azure CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest), and use it to create a new resource group and a Cosmos DB account. But first things first, you need to login to Azure, and select the subscription you want to use"

In [ ]:
!az login

In [ ]:
!az account set -s "your subscription name"

We will now set some variables containing the name of the resource group we will use, the name of the Cosmos DB account to be created, the name for a database and for a collection. Consider that the name for your CosmosDB must be globally unique:

In [ ]:
rg         = "cosmoslab"
location   = "westeurope"

account    = "mycosmos1138"
db_name    = "ecommerce"
coll_name  = "customers"

And now we can create our resource group and Cosmos DB account. When creating the account there some attributes you can defined, such as the type of CosmosDB and API (in this case we will use the SQL API aka DocumentDB), the locations (in this example we will use only one region).

Creating the account will take some minutes, please be patient.

In [ ]:
!az group create -n "$rg" -l "$location"
!az cosmosdb create -g "$rg" -n "$account" --locations "$location"=0 --kind GlobalDocumentDB

## Modify Cosmos DB account

After creating an account, you can modify it, for example to add new read or write regions. In this example we will add a second read region with a failover priority of 1. Note other interesting attributes, such as the default consistency level (session) or different failover properties:

In [ ]:
!az cosmosdb update -g "$rg" -n "$account" --locations "$location"=0 northeurope=1

You can have a look at the help for the `az cosmosdb update` command for more options to update existing Cosmos DB accounts.

In [ ]:
!az cosmosdb update -h

## Authentication

Authentication to a Cosmos DB account works using an account key. There are two account keys for key rotation. You can use the Azure CLI to get the key for an existing account, in this case we get the primary key:

In [ ]:
account_key = !az cosmosdb list-keys -g $rg -n $account --query primaryMasterKey -o tsv
# Per default the output is a list, keep only the first item
account_key = str(account_key[0])

## Initialize the client

You can create databases and collections using the Azure CLI too, or using the Python SDK already. Since this notebook is about Python, let us use the Python SDK already for this.

The first thing we need is to initialize the client, with the Cosmos DB account endpoint (that you can derive from the name, or use the Azure CLI to find out), and the master key that we already got.

In [ ]:
# Option 1: Building the endpoint as a string
cosmosdb_endpoint = "https://{0}.documents.azure.com".format(account)
print(cosmosdb_endpoint)

In [ ]:
# Option 2: Getting the endpoint with the Azure CLI
cosmosdb_endpoint = ! az cosmosdb show -g $rg -n $account --query documentEndpoint -o tsv
cosmosdb_endpoint = str(cosmosdb_endpoint[0])
print(cosmosdb_endpoint)

Before initializaing the client, let us make sure that we have all required Python dependencies:

In [ ]:
!pip install -r requirements.txt

As you can see in the [reference page for the CosmosClient method](https://docs.microsoft.com/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#createdatabase-database--options-none-), other than the URL and the authentication you can supply a [connection policy](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.documents.connectionpolicy?view=azure-python) and the consistency level to use.

The connection policy is especially interesting, since it allows defining attributes such as a list of preferred locations to use (typically the closest geographically to the application), disable SSL verification or retry options, amongst others. Refer to the [reference documentation](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.documents.connectionpolicy?view=azure-python) for more information on the connection policy.

In [ ]:
import azure.cosmos.cosmos_client as cosmos_client

client = cosmos_client.CosmosClient(url_connection=cosmosdb_endpoint, auth={'masterKey': account_key})

## Create Database and Collection

With an initialized client we can now create a database and a collection. Note that you can allocate a performance to the database or to the collection. In this example we will allocate the minimum allowed performance (400 RU/s) to the collection.

See the reference guide for the [createDatabase method](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#createdatabase-database--options-none-) and for the [createContainer method](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#createcontainer--link--collection--options-none-).

Note how when creating a container (called "collection" for document-based schemas) you specify some attributes as part as the collection definition (such as the partition key or the indexing), and others as options (such as the throughput).

Note that in this example we use the property "source" as partition key. This is probably not an ideal partition scheme as we will see during the rest of the notebook, but please do not change this, otherwise some exercises in the notebook will not work as designed. You can find more information about partitions in Cosmos DB [here](https://docs.microsoft.com/azure/cosmos-db/partitioning-overview).

In [ ]:
database_definition = { 'id': db_name }
created_db = client.CreateDatabase(database_definition)
db_link=created_db['_self']

In [ ]:
# Before creating the collection, this code makes sure it does not exist before
# Otherwise it gives out an error, that you could grab in a try/catch block as well
coll_query = "select * from r where r.id = '{0}'".format(coll_name)
coll = list(client.QueryContainers(db_link, coll_query))
if coll:
    print("Collection", coll_name, "already exists")
else:
    collection_definition = {
        'id': coll_name, 
        'indexingPolicy': {'indexingMode': 'consistent'},
        'partitionKey': {
            'paths': [
              '/source'
            ],
            'kind': 'Hash'
        }
    }
    collection_options = { 'offerThroughput': 400 }
    created_collection = client.CreateContainer(created_db['_self'], collection_definition, collection_options)
    print("Collection", coll_name, "has been created")

## Get an existing database or collection

If you are working with an existing database or collection, you just need to retrieve them. The retrieved object will have an s_id field that you can use to verify whether the database and collections where found or not.

In [ ]:
# Get the DB link
db_query = "select * from r where r.id = '{0}'".format(db_name)
db = list(client.QueryDatabases(db_query))
if db:
    db_link = db[0]['_self']
    print("Database", db_name, "found")
    # Uncomment the next line to see the full object
    # print(db[0])

    # If the database has been found, try to find the collection
    coll_query = "select * from r where r.id = '{0}'".format(coll_name)
    coll = list(client.QueryContainers(db_link, coll_query))
    if coll:
        coll_link = coll[0]['_self']
        print("Collection", coll_name, "found:")
        print(" - Indexing policy - Mode:", coll[0]['indexingPolicy']['indexingMode'])
        print(" - Indexing policy - Mode:", coll[0]['indexingPolicy']['automatic'])
        print(" - Conflict resolution policy mode:", coll[0]['conflictResolutionPolicy']['mode'])
        # Uncomment the next line to see the full object
        # print(coll[0])
    else:
        print("Collection", coll_name, "not found :(")
else:
    print("Database", db_name, "not found :(")


Alternatively to the QueryContainers method, you can craft manually a collection link with the database and collection names, and use the method ReadCollection:

In [ ]:
coll_link='dbs/{0}/colls/{1}/'.format(db_name, coll_name)
print("Collection link:", coll_link)
collection = client.ReadContainer(coll_link)
print(collection)

## Deleting an existing database or collection

If you want to play with the different options to create a collection, you can modify most of its attributes, with the notable exception of the partition key. If you need to try the commands above with different options you might have to delete the collection first.

In [ ]:
# Delete only a specific collection
coll_query = "select * from r where r.id = '{0}'".format(coll_name)
coll = list(client.QueryContainers(db_link, coll_query))
if coll:
    coll_link = coll[0]['_self']
    client.DeleteContainer(coll_link)
    print("Collection", coll_name, "has been deleted")
else:
    print("Collection", coll_name, "could not be found")

In [ ]:
# If the DB exists, delete it (including any collections)
db_query = "select * from r where r.id = '{0}'".format(db_name)
db = list(client.QueryDatabases(db_query))
if db:
    db_link = db[0]['_self']
    client.DeleteDatabase(db_link)
    print("Database", db_name, "and all its collections have been deleted")
else:
    print("Database", db_name, "could not be found")

## Updating the RUs of an existing collection

You might want to update the performance (throughput) of your collection under certain circumstances, for example occasionally to perform a performance-intensive operation such as a data import or export, or just because you need more performance on your database on a consistent basis.

To read the current throughput of an existing collection you can use the QueryOffers method, and update it with the ReplaceOffers method.

In [ ]:
coll_query = "select * from r where r.id = '{0}'".format(coll_name)
coll = list(client.QueryContainers(db_link, coll_query))
if coll:
    # We found our collection, get the link and find the offers
    coll_link = coll[0]['_self']
    offer = list(client.QueryOffers('SELECT * FROM c WHERE c.resource = \'{0}\''.format(coll_link)))[0]
    print("Collection", coll_name, "found provisioned with", str(offer['content']['offerThroughput']), "RU/s")
else:
    print("Collection", coll_name, "not found")

In [ ]:
new_offer = offer
new_offer['content']['offerThroughput'] += 50
throughput = new_offer['content']['offerThroughput']
if (throughput >= 400) and (throughput <= 100000) and ((throughput % 100) == 0): 
    offer = client.ReplaceOffer(offer['_self'], new_offer)
else:
    print(throughput, "is not a valid throughput for Cosmos DB")

## Create documents

You can use the Python SDK to create new documents in our collection. For this we will use some sample documents, that are used as well in the [EDX course for Cosmos DB](https://courses.edx.org/courses/course-v1:Microsoft+DAT237x+2T2017/course/). In that course these documents are used to illustrate the .NET API, we will use them here for the Python SDK.

The function insert_item supports a "pretrigger" argument, please ignore it for the time being (we will come back to it later in this tutorial).

In [ ]:
# We will need these throughout the notebook
import os, json

In [ ]:
# Single add
def insert_item(client, coll_link, item, pretrigger=None):
    options={}
    if pretrigger:
        options['preTriggerInclude'] = pretrigger
    client.CreateItem(coll_link, item, options)

# Add all documents in a specific folder
def insert_items_batch(client, coll_link, data_dir):
    jsonpath = os.path.join(os.getcwd(), data_dir)
    counter=0
    for file in os.listdir(jsonpath):
        if file[-5:] == ".json":
            #print("Found json file", file)
            f=open(os.path.join(jsonpath, file), 'r')
            item_json = f.read()
            item = json.loads(item_json)
            insert_item(client, coll_link, item)
            counter += 1
        else:
            print("Non-JSON file found:", file)
    print(counter, "documents added")

In [ ]:
insert_items_batch(client, coll_link, 'data')

## Querying documents

Now that we have some data in the collection, we can start running some queries. Let us start with something simple. As you can see, the send_query function supports defining query options, we will see those in a minute. See the reference documentation for the [QueryItems](https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#queryitems-database-or-container-link--query--options-none--partition-key-none-) method for more information.

In the example below, for output clarity the send_query function has an optional toggle that can be used to hide the actual results, and print only the number of matching documents, as well as the RU/s consumed by this particular query. Note how the consumed RU's are obtained by looking at a specific field of the client object, that contains the consumed RU's for the last operation.

You can find more examples of queries for Azure Cosmos DB [here](https://docs.microsoft.com/azure/cosmos-db/how-to-sql-query).

In [ ]:
# Auxiliary function that we will use to send a query to a collection
def send_query(client, coll_link, query, options=None, show_results=True):
    docs = list(client.QueryItems(coll_link, query, options))
    for doc in docs:
        if show_results:
            print(json.dumps(doc, indent=4, sort_keys=True))
    print(len(docs), 'items found,', client.last_response_headers['x-ms-request-charge'], 'RU/s consumed')

In [ ]:
# Query with partition ID
customer_id='09d2bb28e9c54bc581492d542789f2ad'
source='other'
query = 'SELECT * FROM customers WHERE customers.source=\'{0}\' AND customers.id=\'{1}\''.format(source, customer_id)
send_query(client, coll_link, query, show_results=False)

Note that if you leave out the source in the query, as the example below, Cosmos DB is forced to look into every partition. Hence, if you do not specify in the options that this is to be a cross-partition query, you will get an error message.

In [ ]:
# Query without partition ID
customer_id='09d2bb28e9c54bc581492d542789f2ad'
options = {} 
options['enableCrossPartitionQuery'] = True
query = 'SELECT * FROM customers WHERE customers.id=\'{0}\''.format(customer_id)
send_query(client, coll_link, query, options=options, show_results=False)

As you might have seen in the code for send_query, you can send some options along with the query, that will modify the way in which the query is executed. For example, we can enable cross-partition queries (to be able to select all records) and set the maximum item count to 2:

In [ ]:
# Simple query with options
query = 'SELECT * FROM customers'
options = {} 
options['enableCrossPartitionQuery'] = True
options['maxItemCount'] = 2
send_query(client, coll_link, query, options, show_results=False)

More complex queries are supported using the SQL dialect of Cosmos DB:

In [ ]:
# Query showing projection and filtering
query='''SELECT {
     "full-name": customers.name,
     "contact-details": {
        "phone": customers.phone,
        "address": customers.address
    },
     "employment": {
        "employer": customers.company,
        "work-email": customers.email
     }
} AS person
FROM customers
WHERE customers.source = "retail-location"'''
options = {} 
options['enableCrossPartitionQuery'] = True
send_query(client, coll_link, query, options, show_results=False)

## Triggers

Triggers are activites that can be executed before ("Pre" triggers) or after ("Post" triggers) data being written or updated in Cosmos DB. Triggers are written in JavaScript, which makes working with JSON very easy. In this repository we have a trigger that adds a 'department' property to a document before adding it to the collection, in case that property did not exist.

In [ ]:
import os
trigger_filename = os.path.join(os.getcwd(), 'triggers', 'validateDepartmentExists.js')
f=open(trigger_filename, 'r')
trigger_code = f.read()
print(trigger_code)

In [ ]:
import azure.cosmos.documents as documents

# Create new trigger from file
def create_trigger(client, coll_link, filename):
    with open(filename) as file:
        file_contents = file.read()
    trigger_name = os.path.splitext(os.path.basename(filename))[0]
    trigger_definition = {
        'id': trigger_name,
        'serverScript': file_contents,
        'triggerType': documents.TriggerType.Pre,
        'triggerOperation': documents.TriggerOperation.All
    }
    trigger = client.CreateTrigger(coll_link, trigger_definition)

# List existing defined triggers in the collection
def query_triggers(client, coll_link):
    trigger_query = 'select * from r'
    triggers = list(client.QueryTriggers(coll_link, trigger_query))
    for trigger in triggers:
        print(json.dumps(trigger, indent=4, sort_keys=True))

    
trigger_filename = os.path.join(os.getcwd(), 'triggers', 'validateDepartmentExists.js')
create_trigger(client, coll_link, trigger_filename)
query_triggers(client, coll_link)

We can now add a new document without the 'department' property, and verify that the property was added by the Trigger. For that we will use a helper function that generates a customer with a random ID:

In [ ]:
import random, string

def get_random_item():
    random_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=16))
    itemToCreate = {
        'firstName': 'Sample',
        'lastName': 'Person',
        'id': random_id,
        'source': 'random'
    }
    return itemToCreate

In [ ]:
random_item = get_random_item()
print('We will send an item without the \'department\' property:')
print(random_item)
# Note that we do specify the ID of the Pre trigger to execute
insert_item(client, coll_link, random_item, pretrigger='validateDepartmentExists')
print("And now the item actually created should have the \'department\' property, added by the trigger:")
query = 'SELECT * FROM customers WHERE customers.id = \'{0}\''.format(random_item['id'])
options = {} 
options['enableCrossPartitionQuery'] = True
send_query(client, coll_link, query, options)

## User-Defined Functions (UDF)

You can define your own functions that can be used in queries. In this section we are going to create a user-defined function (UDF) that converts from US dollars to euros. The function is in the "udf" directory of this repository. First we will add the function to the collection: 

In [ ]:
# Create UDF from a file with JS code
def create_udf(client, coll_link, filename):
    udf_name = os.path.splitext(os.path.basename(filename))[0]
    if get_udf(client, coll_link, udf_name):
        print("UDF", udf_name, "already exists, you might want to update it instead")
    else:
        with open(filename) as file:
            file_contents = file.read()
        udf_definition = {
            'id': udf_name,
            'serverScript': file_contents,
        }
        return client.CreateUserDefinedFunction(coll_link, udf_definition)

# Get UDF by name
def get_udf(client, coll_link, udf_name):
    udf_query = 'select * from r where r.id ="' + udf_name + '"'
    udfs = list(client.QueryUserDefinedFunctions(coll_link, udf_query))
    if len(udfs) > 0:
        return udfs[0]
    else:
        return None
    
create_udf(client, coll_link, os.path.join(os.getcwd(), 'udf', 'convertToEuro.js'))


You can now verify that the UDF has been properly created 

In [ ]:
def query_udfs(client, coll_link):
    udf_query = 'select * from r'
    udfs = list(client.QueryUserDefinedFunctions(coll_link, udf_query))
    for udf in udfs:
        print(json.dumps(udf, indent=4, sort_keys=True))

# Verify the function is created
query_udfs(client, coll_link)

In order to modify the function (for example if you change the original file), you can use the ReplaceUserDefinedFunction method:

In [ ]:
# Update existing UDF with a file
def update_udf(client, coll_link, udf_name, filename):
    udf_query = 'SELECT * from r WHERE r.id = "' + udf_name + '"'
    udfs = list(client.QueryUserDefinedFunctions(coll_link, udf_query))
    if len(udfs) > 0:
        udf_link = udfs[0]['_self']
        with open(filename) as file:
            file_contents = file.read()
        udf_definition = {
                    'id': udf_name,
                    'body': file_contents,
                }
        client.ReplaceUserDefinedFunction(udf_link, udf_definition)
    else:
        print("UDF", udf_name, "not found!")

update_udf(client, coll_link, 'convertToEuro', os.path.join(os.getcwd(), 'udf', 'convertToEuro.js'))

Now we are ready to use the function inside of a simple query. Note the higher RU consumption (you can try to execute the same query without the UDF to compare):

In [ ]:
query = 'SELECT TOP 1 c.name, c.balance, udf.convertToEuro(c.balance) AS balanceEUR FROM customers c WHERE c.source = "word-of-mouth"'
options = {} 
options['enableCrossPartitionQuery'] = True
send_query(client, coll_link, query, options=options)

## Stored procedures

The SQL API of Cosmos DB supports stored procedures, that can be used for multiple objectives, such as for example to provide transaction semantics in Cosmos DB. You can find more information in the Cosmos DB documentation about [how to write stored procedures](https://docs.microsoft.com/en-us/azure/cosmos-db/how-to-write-stored-procedures-triggers-udfs#stored-procedures).

In this notebook we will use a stored procedure to perform a transfer between the balance of two customers.

In [ ]:
# Adding the stored procedure
def create_sproc(client, s_coll, filename):
    sproc_name = os.path.splitext(os.path.basename(filename))[0]
    if get_sproc(client, coll_link, sproc_name):
        print("Stored Procedure", sproc_name, "already exists, you might want to update it instead")
    else:
        with open(filename) as file:
            file_contents = file.read()
        sproc_definition = {
                    'id': sproc_name,
                    'serverScript': file_contents,
                }
        sproc = client.CreateStoredProcedure(coll_link, sproc_definition)

# Get stored procedure by name
def get_sproc(client, coll_link, sproc_name):
    sproc_query = 'select * from r where r.id ="' + sproc_name + '"'
    sprocs = list(client.QueryStoredProcedures(coll_link, sproc_query))
    if len(sprocs) > 0:
        return sprocs[0]
    else:
        return None

create_sproc(client, coll_link, os.path.join(os.getcwd(), 'storedproc', 'TransferBalance.js'))

You can verify that the stored procedure has been created correctly:

In [ ]:
# Print the list of existing store procedures
def query_sproc(client, coll_link):
    sp_query = 'select * from r'
    sprocs = list(client.QueryStoredProcedures(coll_link, sp_query))
    for sproc in sprocs:
        print(json.dumps(sproc, indent=4, sort_keys=True))

query_sproc(client, coll_link)

If you need to change the stored procedure, you can use this method:

In [ ]:
# To update an existing stored procedure
def update_sproc(client, coll_link, sproc_name, filename):
    sp_query = 'SELECT * from r WHERE r.id = "' + sproc_name + '"'
    sprocs = list(client.QueryStoredProcedures(coll_link, sp_query))
    if len(sprocs) > 0:
        sproc_link = sprocs[0]['_self']
        with open(filename) as file:
            file_contents = file.read()
        sproc_definition = {
                    'id': sproc_name,
                    'body': file_contents,
                }
        client.ReplaceStoredProcedure(sproc_link, sproc_definition)
    else:
        print("Stored procedure", sproc_name, "not found!")

update_sproc(client, coll_link, 'TransferBalance', os.path.join(os.getcwd(), 'storedproc', 'TransferBalance.js'))

Now we will define a function to execute the stored procedure, and send some balance from one customer to another:

In [ ]:
# Execute stored procedure
def execute_sproc_transferBalance(partition_value, id_from, id_to, amount):
    sp_name = 'TransferBalance'
    sp_query = 'SELECT * from r WHERE r.id = "' + sp_name + '"'
    sprocs = list(client.QueryStoredProcedures(coll_link, sp_query))
    if len(sprocs) > 0:
        sproc_link = sprocs[0]['_self']
        options = {
            'setScriptLoggingEnabled': True,
            'partitionKey': partition_value
        }
        sproc_response = client.ExecuteStoredProcedure(sproc_link, params=[id_from, id_to, amount], options=options)
        # The response is the ID of the created resource, not a dictionary as the doc seems to suggest
        # https://docs.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos.cosmos_client.cosmosclient?view=azure-python#executestoredprocedure-sproc-link--params--options-none-
        print(type(sproc_response), ':', sproc_response)
    else:
        print("Stored procedure", sp_name, "not found!")


We can finally run our transaction using the stored procedure. The following code has two customer IDs in the same partition `word-of-mouth`. When you run the code as-is, you should see that $100 has been transfered from the customer1 (Benton Cooley) to the customer2 (Warren Holman).

However, if you uncomment the third line (`id2="09d2bb28e9c54bc581492d542789f2ad"`) you will find receive an error message: `Unable to find second customer`. This is because transactions in Azure Cosmos DB are limited to a single partition, and the ID in this third line corresponds to Giliam Greener, with source='other' and hence in a different subscription.

In [ ]:
id1="1240d84bd5f44074b36bd1977bc9062c"
id2="5c89c87d34e64f37a3c8fae3fb86d8fc"
# id2="09d2bb28e9c54bc581492d542789f2ad"
amount_to_transfer = 100
query = 'SELECT c.name, c.source, c.balance, c.id FROM customers c WHERE c.id IN ("{0}", "{1}")'.format(id1, id2)
options = {} 
options['enableCrossPartitionQuery'] = True
send_query(client, coll_link, query, options=options)
execute_sproc_transferBalance('word-of-mouth', id1, id2, amount_to_transfer)
send_query(client, coll_link, query, options=options)

## Change feed

The change feed is an extremely interesting feature of Cosmos DB, since it enables many usage scenarios such as data replication, triggering serverless computing such as Azure Functions, or cost-effective data deletions, to name a few. You can find more information about Azure Cosmos DB change feed [here](https://docs.microsoft.com/azure/cosmos-db/change-feed).

<img src="figures/changefeedoverview.png" width="700"/>

The following code shows how to get the change feed from the creation of the database, and how to use the etag header to ask for changes occurred since the last query to the feed, which after creating one document, should be only one.

In [ ]:
def get_change_feed(client, coll_link, continuation=None, show_results=True):
        options = {}
        #options['partitionKeyRangeId'] = ''
        if continuation:
            options['continuation'] = continuation
        else:
            options["startFromBeginning"] = True
        response = client.QueryItemsChangeFeed(coll_link, options)
        if show_results:
            i=0
            for doc in response:
                print(doc)
                i += 1
            count = i
        else:
            count = len(tuple(response))
        print(count, "change feed items found, continuation", client.last_response_headers['etag'])
        
get_change_feed(client, coll_link, show_results=False)

In [ ]:
continuation=client.last_response_headers['etag']
insert_item(client, coll_link, get_random_item())
get_change_feed(client, coll_link, continuation=continuation, show_results=True)

You can have a look at [this sample code](https://github.com/Azure/azure-cosmos-python/blob/master/samples/ChangeFeedManagement/Program.py) for more on handling the Azure Cosmos DB Change feed with Python.

## Cleanup

In order to save costs you can delete the resource group that was created at the beginning of this exercise via the Azure CLI (or any other method). Please be careful with this command, since it will delete the resource group as well as every resource inside, including the Cosmos DB account and all databases and collections.

In [ ]:
!az group delete -n $rg -y --no-wait